# Import statements

In [2]:
%matplotlib notebook

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math

# Loading of data

In [14]:
data = np.genfromtxt('dataset', delimiter =',')
info = data[:,1]
plt.plot(info[800:2000])

#  Processing (Log and Standarization)

In [61]:
shiftval = np.abs(np.min(info))+1
print(shiftval)
shifted = info + shiftval
logged = np.log10(shifted)

Lmean = np.mean(logged)
Lstd = np.std(logged)
normed = (logged - Lmean) / Lstd

491961.47


#  Data Separation

In [132]:
dataESN = normed[:]
#initialize the reserviour parameters
_input = 1
output = 1
units = 1000

kfold = 6
Prediction = 2400
New_train = dataESN[0:-Prediction]
New_test = dataESN[-Prediction:]
New_teach = dataESN[168:-Prediction+168]
wash = 400
print(len(New_train), len(New_test), len(New_teach))

8568 2400 8568


# Initializing ESN

In [133]:
#initialize the reserviour parameters
_input = 1
output = 1
units = 1000

np.random.seed(42)
#reservoir parameters
Win = (np.random.rand(units,_input)-0.5) * 1
W = np.random.rand(units,units)-0.5 
bias = np.random.rand(units,1)-0.5 

#calculating the spectral radius
sp_radius = np.max(np.abs(np.linalg.eig(W)[0]))
print(sp_radius)
W *= 1/sp_radius

#scaling factors
Winsf = 2.0
Wsf = 1.5
bsf = 0.48
alpha = 0.028
#scale the matrices
Win *= Winsf
W *= Wsf
bias *= bsf

# Cross Validation

In [134]:
NRMSE_vec = np.zeros(kfold)
MAPE_vec = np.zeros(kfold)
Nmax = len(New_train)

for k in range(kfold):
    aval = (k) * int(np.floor(Nmax/kfold))
    bval = (k+1) * int(np.floor(Nmax/kfold))
    print('ROUND: ', k)
    
    input_test = New_train[aval:bval]
    teach = np.hstack((New_teach[0:aval],New_teach[bval:Nmax]))
    train = np.hstack((New_train[0:aval],New_train[bval:Nmax]))
    
    trainlen = np.shape(train)[0]
    testlen = np.shape(input_test)[0]
    

    # data collection matrix
    SCM = np.zeros((_input+units,trainlen), dtype=np.float32)
    p = np.zeros((1+units), dtype=np.float32)
    x = np.zeros((units,), dtype=np.float32)
    
    # training phase
    for j in range(trainlen):
        u = train[j]
        p[0] = u
        x = np.tanh(np.dot(Win,u) + np.dot(W,x) + bias)
        p[1:] = x
        SCM[:, j] = p
  
    teach =teach[wash:]
    SCM = SCM[:, wash:]
    Wout = np.dot(np.dot(teach.T, SCM.T), np.linalg.inv(np.dot(SCM,SCM.T) + alpha*np.eye(units+_input)))

    # Prediction phase
    Y = np.zeros(testlen)
    u = input_test[0]
    for t in range(testlen-1):
        x = np.tanh(np.dot(Win,u) + np.dot(W,x) + bias)
        p[0] = u
        p[1:] = x
        y = np.dot(Wout, p)
        Y[t] = y
        u = input_test[t+1]
    # Error calculation
    input_test = input_test[1:]
    Y = Y[0:testlen-1]
    Error = input_test - Y
    MSE = np.mean(np.power(Error, 2))
    varD = np.mean(np.power((input_test - np.mean(input_test)), 2))
    NRMSE = math.sqrt(MSE/varD)
    NRMSE_vec[k] = NRMSE
    MAPE = np.mean(np.abs((input_test - Y)/Y))
    MAPE_vec[k] = MAPE
    
    
meanNRMSE = np.mean(NRMSE_vec)
print('NRMSEmean: ', meanNRMSE, '\nNrmse: ', NRMSE_vec)
meanMAPE = np.mean(MAPE_vec)
print('MAPEmean: ', meanMAPE, '\nMAPE: ', MAPE_vec)

ROUND:  0
ROUND:  1
ROUND:  2
ROUND:  3
ROUND:  4
ROUND:  5
NRMSEmean:  0.545039029833 
Nrmse:  [ 0.60857832  0.49489903  0.50962825  0.53189226  0.69696164  0.42827467]
MAPEmean:  3.67733040556 
MAPE:  [ 3.04154555  8.8235768   1.7770876   2.21780495  3.22255339  2.98141414]


# Final Testing

In [136]:
finaltestlen = New_test.shape[0]

ST= np.zeros((_input+units,finaltestlen), dtype=np.float32)
p = np.zeros((1+units), dtype=np.float32)
x = np.zeros((units,), dtype=np.float32)
    
for f in range(finaltestlen):
        u = New_test[f]
        p[0] = u
        x = np.tanh(np.dot(Win,p[0]) + np.dot(W,x) + bias)
        p[1:] = x
        ST[:, f] = p

YoutT = np.dot(Wout, ST)
YoutT.shape[0]

# Final Error Calculation

In [139]:
Error = New_test - YoutT
MSE = np.mean(np.power(Error, 2))
varD = np.mean(np.power((New_test - np.mean(New_test)), 2))
NRMSE = math.sqrt(MSE/varD)
MAPE = np.mean(np.abs((New_test - YoutT)/YoutT))
print(NRMSE)
print(MAPE)

0.3471791982218397
1.91335912317
